<a href="https://colab.research.google.com/github/BernieT7/AI/blob/main/%E6%8A%95%E8%B3%87%E7%90%86%E8%B2%A1%E9%A1%A7%E5%95%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai
!pip install python-dotenv
!pip install gradio==3.48.0
!pip install pytubefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━

In [3]:
from openai import OpenAI
from dotenv import dotenv_values
config = dotenv_values('.env')
client = OpenAI(api_key=config["API_KEY"])

In [4]:
from pytubefix import YouTube, Playlist

In [5]:
def get_audio_text(audio_path, title):
  audio = open(audio_path, "rb")               # 讀取音檔
  res = client.audio.transcriptions.create(          # 利用audioAPI將音檔轉換為文字
      model="whisper-1",
      file=audio,
      prompt=title                      # 以影片標題引導模型的指示
  )
  return res.text                        # 回傳影片音檔文字

In [6]:
def get_video_title_text(video_url, audio_name):
  video = YouTube(video_url)                  # 創建YT影片
  stream = video.streams.filter(only_audio=True).first()    # 取得YT影片音檔
  audio_path = stream.download(filename=audio_name)       # 將音檔下載至本地端
  text = get_audio_text(audio_path, video.title)        # 取得文字
  return video.title, text                   # 回傳影片標題及文字

In [24]:
def get_playlist_info(playlist_url):
  playlist = Playlist(playlist_url)                  # 取得playlist裡所有影片的網址
  videos_info = {}
  for idx, video_url in enumerate(playlist):
    title, text = get_video_title_text(video_url, f"{idx}.mp3")  # 取得所有影片標題及文字
    videos_info[title] = text                    # 存放所有影片標題及文字
  return videos_info                          # 回傳所有影片標題及文字

In [ ]:
playlist_url = "https://www.youtube.com/playlist?list=PLrZrfGLGySzcZoVhb4idy5B0XI25ZhnF7" # 引用YouTuber柴鼠兄弟的playList
playlist_info = get_playlist_info(playlist_url)

In [11]:
import pandas as pd

In [ ]:
df = pd.DataFrame(list(playlist_info.items()), columns=["title", "text"])
df.to_csv("video_text.csv", index=False)

In [30]:
df = pd.read_csv("video_text.csv")
df

,title,text
0,銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,夯翻鼠FQ1 投資理財 大家好我是柴 大家好我是鼠 歡迎收看第一集夯翻鼠FQ1 我們每一集用...
1,存股一定打敗銀行定存？常被忽略的三個弱點和一個關鍵... | 夯翻鼠FQ2 投資理財,夯翻鼠FQ2 投資理財 大家好我是柴 大家好我是鼠 歡迎收看第二集夯翻鼠FQ 上一集我們了解...
2,匯率霧傻傻？先換起來放？錢被銀行默默吃掉的三個秘密 | 夯翻鼠FQ3 投資理財,夯翻鼠FQ3 投資理財 大家好我是鼠 歡迎收看第三集夯翻鼠FQ 有很多人出國換錢的經驗 是來...
3,2018存股怎麼選？新手秒懂的-放山雞選股法(概念篇) | 夯翻鼠FQ4 投資理財,夯翻鼠FQ4 投資理財 大家好我是財 大家好我是鼠 歡迎收看第四集夯翻鼠FQ 之前我們在上一...
4,不意外！台股平均20年殖利率排行前三名是他們⋯ 放山雞選股法(網站實作篇) | 夯翻鼠FQ5...,夯翻鼠FQ5 投資理財 大家好我是柴 大家好我是鼠 歡迎收看第五集夯翻鼠FQ 在上一支超值選...
...,...,...
72,【入坑實測】參加ETF下市清算結果出乎意料🤓 下市規定有二種不要搞錯 00742新光內需收益...,大家好我是柴 我是蘇 派克老闆在這裡 歡迎收看第73集夯翻鼠FQ 很多ETF投資人經常會問 ...
73,巴菲特的波克夏大買台積電ADR🍗 美國存託憑證和ETF哪裡像？ | 夯翻鼠FQ74 TSMC,大家好我是X 我是Sue 潘凱老闆沒有來 歡迎收看第74集夯翻鼠FQ 要說到近期台股最熱的話...
74,⟪債券ETF完整攻略⟫一張表選完2023人氣美債公司債ETF🏦 存債勝存股三大免稅費優勢 |...,大家好我是柴 我是佐久 歡迎收看第76集夯翻鼠FQ 講到台灣的ETF 常聽到的不管是0050...
75,報稅起跑！2023綜合所得稅扣除額&級距全圖解 上班族存股ETF配息節稅攻略 | 夯翻鼠FQ76,大家好我是柴 我是鼠兒 派凱老闆 歡迎收看第76集夯翻鼠FQ 更正一下上一集債券是75 這一...


In [31]:
def split_text(all_text, title):                # 由於模型能容納的token數有限，原本一段影片的總token數遠超模型限制，所以要將每句話分割出來，分次embedding轉換
  text_list = all_text.split(' ')              # 把每句話分開
  text = title                        # 分開後前面加上title也就是後面引導模型的指示
  new_text_list = []                     # 接收加上title後的新text_list
  for idx, i in enumerate(text_list):            # 利用for loop昨上述任務
    text += f",{i}"
    if (idx+1)%50==0 or idx==len(text_list)-1:       # 每50句話一組或是做到最後一組就將結果存入new_text_list
      new_text_list.append(text)
      text = title
  return new_text_list

In [32]:
split_text_list = []
for idx, text in enumerate(df["text"].values):        # 把playlist的所有影片分割
  split_text_list += split_text(text, df["title"][idx])

In [33]:
df = pd.DataFrame(split_text_list, columns=['split_text_list'])
df

,split_text_list
0,"銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,夯翻鼠FQ1,投資理..."
1,"銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,如果定存時間只設定成..."
2,"銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,我們用最直觀的概念來..."
3,"銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,還是會把殖利率的標準..."
4,"銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,這是一個關於投資理財..."
...,...
430,純股族退稅8萬有可能嗎？三條線求出股利所得退稅最大值 一張表查出參加除權息超過幾萬要繳稅 |...
431,純股族退稅8萬有可能嗎？三條線求出股利所得退稅最大值 一張表查出參加除權息超過幾萬要繳稅 |...
432,純股族退稅8萬有可能嗎？三條線求出股利所得退稅最大值 一張表查出參加除權息超過幾萬要繳稅 |...
433,純股族退稅8萬有可能嗎？三條線求出股利所得退稅最大值 一張表查出參加除權息超過幾萬要繳稅 |...


In [34]:
def get_embedding(text):
  res = client.embeddings.create(            # 利用模型將文字向量化
      model="text-embedding-ada-002",
      input=text                    # 輸入欲轉換文字
  )
  return res.data[0].embedding              # 回傳轉換結果

In [35]:
split_text_embeddings = [get_embedding(i) for i in df["split_text_list"]]   # 將每段文字轉換成向量

In [36]:
df["embeddings"] = split_text_embeddings         # 創建新的column，將向量化後的結果展現出來
df

,split_text_list,embeddings
0,"銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,夯翻鼠FQ1,投資理...","[0.008625390939414501, -0.007851073518395424, ..."
1,"銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,如果定存時間只設定成...","[0.005137754138559103, 0.0012010129867121577, ..."
2,"銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,我們用最直觀的概念來...","[-0.0012613272992894053, 0.0027926538605242968..."
3,"銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,還是會把殖利率的標準...","[0.008714095689356327, -0.009046598337590694, ..."
4,"銀行定存怎麼算利息錢？一次搞懂利率&殖利率 | 夯翻鼠FQ1 投資理財,這是一個關於投資理財...","[0.008075352758169174, -0.013166887685656548, ..."
...,...,...
430,純股族退稅8萬有可能嗎？三條線求出股利所得退稅最大值 一張表查出參加除權息超過幾萬要繳稅 |...,"[-0.02336304262280464, -0.011888962239027023, ..."
431,純股族退稅8萬有可能嗎？三條線求出股利所得退稅最大值 一張表查出參加除權息超過幾萬要繳稅 |...,"[-0.013771829195320606, -0.010248202830553055,..."
432,純股族退稅8萬有可能嗎？三條線求出股利所得退稅最大值 一張表查出參加除權息超過幾萬要繳稅 |...,"[-0.01556857954710722, -0.011319681070744991, ..."
433,純股族退稅8萬有可能嗎？三條線求出股利所得退稅最大值 一張表查出參加除權息超過幾萬要繳稅 |...,"[-0.011633334681391716, -0.023730972781777382,..."


In [37]:
from typing import List
from scipy import spatial
import numpy as np



def distances_from_embeddings(
    query_embedding: List[float],
    embeddings: List[List[float]],
    distance_metric="cosine",
) -> List[List]:
    """Return the distances between a query embedding and a list of embeddings."""
    distance_metrics = {
        "cosine": spatial.distance.cosine,
        "L1": spatial.distance.cityblock,
        "L2": spatial.distance.euclidean,
        "Linf": spatial.distance.chebyshev,
    }
    distances = [
        distance_metrics[distance_metric](query_embedding, embedding)
        for embedding in embeddings
    ]
    return distances


def indices_of_nearest_neighbors_from_distances(distances) -> np.ndarray:
    """Return a list of indices of nearest neighbors from a list of distances."""
    return np.argsort(distances)

In [43]:
def finance(question):
  question_embeddings = get_embedding(question)
  dist = distances_from_embeddings(question_embeddings, df["embeddings"])
  nearest_idx = indices_of_nearest_neighbors_from_distances(dist)
  nearest_text = ""
  for i in range(2):
    nearest_text += df["split_text_list"][nearest_idx[i]] + '\n'

  prompt = f"""
  你是我的投資理財顧問，請根據以下內容回答此問題:{question}
  如果沒有100%確定，就回答'我不知道'

  ###
  內容:
  {nearest_text}
  ###

  """
  res = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      max_tokens=500,
      temperature=0
  )
  return res.choices[0].text

In [44]:
import gradio as gr

demo = gr.Interface(
    fn=finance,
    inputs="text",
    outputs="text",
    title="投資理財顧問",
    description="輸入您的問題:",
    allow_flagging="never"
)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://9d3dd1fca12ef97ccd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9d3dd1fca12ef97ccd.gradio.live
